# MCP and AgentCore Authentication with Okta 

This notebook provides a comprehensive guide to deploying and testing an MCP (Model Context Protocol) Agent on AWS Bedrock AgentCore with Okta OAuth authentication.

## Overview

The MCP Agent integrates external data sources through the Model Context Protocol, enabling AI agents to access real-time information from AWS services like Pricing and Documentation APIs. This setup demonstrates secure authentication flows using Okta OAuth 2.0 with Bedrock Agent Core.

### Architecture Overview

```
┌────────┐    1. Credentials    ┌───────┐    2. JWT Token    ┌────────┐
│ Client │ ──────────────────► │ Okta  │ ─────────────────► │ Client │
└────────┘                     └───────┘                    └───┬────┘
                                                                │
                                                                │ 3. Bearer JWT
                                                                ▼
┌────────┐    6. Response    ┌──────────────┐    4. Invoke    ┌───────────┐
│ Client │ ◄──────────────── │ Bedrock      │ ──────────────► │ MCP Agent │
└────────┘                  │ Agent Core   │ ◄──────────────── └───┬───────┘
                            └──────────────┘    5. Response        │
                                                                   │ MCP stdio
                                                                   ▼
                                                            ┌─────────────┐
                                                            │ MCP Servers │
                                                            │ (Pricing,   │
                                                            │  Docs)      │
                                                            └─────────────┘
```



## Prerequisites

- AWS Account with Bedrock Agent Core access
- Okta Developer Account
- Python 3.9+
- AWS CLI configured
- Docker (for containerized deployment)


## Step 1: Setting up Okta's IDP

Before we can configure our AgentCore Runtime with Okta authentication, we need to set up Okta as our Identity Provider. This section will guide you through creating an Okta tenant, configuring an application, and setting up the necessary users and claims.

### 1.1 Create Okta Developer Account

If you don't already have an Okta account, browse to https://developer.okta.com/signup/ and select "Sign up for Integrator Free Plan" to sign up.

### 1.2 Add a Test User

1. Login to your Okta account.
2. Select **Directory**, then **People** and click **Add person**.

   <figure>
       <img src="images/9.png">
   </figure>

3. Fill in the form:
   - For **Activation**, select **Activate now**.
   - Check **I will set password** and set a password for the user.
   - Uncheck **User must change password on first login**.
   - Click **Save**.

   <figure>
       <img src="images/10.png">
   </figure>

### 1.3 Create Application Integration

4. Select **Applications**, then click **Create App Integration**.

   <figure>
       <img src="images/1.png">
   </figure>

5. For the sign-in method, select **OIDC - OpenID Connect**, then select **Web Application** for the application type.

   <figure>
       <img src="images/2.png">
   </figure>

6. Configure the application:
   - For the App integration name enter **Travel Assistant**
   - Select **Authorization Code** for the grant type.

   <figure>
       <img src="images/3.png">
   </figure>

   - Leave the sign-in and sign-out redirect URIs as is.

   <figure>
       <img src="images/4.png">
   </figure>

   - Under assignments, select **Allow everyone in your organization to access**, then leave **Enable immediate access** checked. Click **Save**.

   <figure>
       <img src="images/5.png">
   </figure>

   - Copy the **Client ID** and **Secret** for later use.

   <figure>
       <img src="images/6.png">
   </figure>

### 1.4 Configure Authorization Server

7. In the left-hand side menu, select **Security**, then **API**, and click the name of your authorization server.

   <figure>
       <img src="images/7.png">
   </figure>

   - Copy the **Audience** and save it for later use.
     > **Note**: The default **Audience** was changed in this example. It is recommended to add a new authorization server if you plan to change the audience so that other apps are not affected.
   
   - Click **Claims** and add the following **client_id** and **scope** claims.

   <figure>
       <img src="images/8.png">
   </figure>

### 1.5 Collect Configuration Values

After completing the Okta setup, you should have the following values:

- **OKTA_CLIENT_ID**: Application's Client ID from General tab
- **OKTA_CLIENT_SECRET**: Application's Client Secret from General tab  
- **OKTA_AUDIENCE**: Audience (e.g., `testagentcore`)
- **OKTA_TOKEN_URL**: Your Okta domain + `/oauth2/default/v1/token`
- **OKTA_DISCOVERY_URL**: Your Okta domain + `/oauth2/default/.well-known/openid-configuration`

Keep these values handy as you'll need them in the next steps.


## Step 2: Environment Setup

First, let's set up the development environment and install required dependencies.

In [ ]:
# Create and activate virtual environment
!python -m venv .venv
!source .venv/bin/activate

For this code to run, the Strands Agents modules need to be installed in the Python environment.

Add the Strands Agents modules, AgentCore SDK, and AgentCore starter toolkit to the dependency file and save it as **requirements.txt**:

In [ ]:
%%writefile requirements.txt
strands-agents
strands-agents-tools
bedrock-agentcore
bedrock-agentcore-starter-toolkit

In [ ]:
# Install required packages
!pip install -r requirements.txt

In [ ]:
# Verify installations
try:
    import bedrock_agentcore
    import strands
    print("✅ All packages installed successfully")
    print("✅ bedrock-agentcore: imported")
    print("✅ strands: imported")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please ensure all packages are installed correctly")

### Step 3: Configure Environment Variables

Now that you have completed the Okta setup, configure the environment variables with the values you collected:

In [ ]:
import os

# Okta Configuration - Replace with your values
os.environ['OKTA_CLIENT_ID'] = 'YOUR CLIENT ID VALUE'
os.environ['OKTA_CLIENT_SECRET'] = 'YOUR CLIENT SECRET VALUE'
os.environ['OKTA_AUDIENCE'] = 'YOUR OKTA AUDIENCE'
os.environ['OKTA_TOKEN_URL'] = 'https://your.okta.com/oauth2/default/v1/token'
os.environ['OKTA_DISCOVERY_URL'] = 'https://your.okta.com/oauth2/default/.well-known/openid-configuration'

print("✅ Okta environment variables configured")

## Step 4: Create MCP Client and configure MCP server Implementation

Create the MCP Agent that integrates with AWS Pricing and Documentation APIs.

In [ ]:
%%writefile my_mcp_agent.py
#!/usr/bin/env python3
"""
AWS Solutions Architect - MCP Agent with Pricing and Documentation Integration
"""

from bedrock_agentcore import BedrockAgentCoreApp
from mcp import StdioServerParameters, stdio_client
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient

# MCP Servers - Cost Explorer and Documentation
aws_pricing_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-pricing-mcp-server@latest"]
        )
    )
)

aws_docs_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    temperature=0.7,
)

SYSTEM_PROMPT = """
You are an expert AWS Solutions Architect specializing in database migrations.
Create comprehensive migration documentation following AWS Well-Architected Framework.
Use AWS documentation for best practices and pricing analysis for TCO calculations.
"""

app = BedrockAgentCoreApp()

@app.entrypoint
def invoke(payload):
    """Handler for agent invocation"""
    prompt = payload.get("prompt", "Please provide a prompt for the migration guide")
    
    print("Initializing MCP servers...")
    with aws_pricing_client, aws_docs_client:
        print("Getting available tools...")
        all_tools = (
            aws_pricing_client.list_tools_sync() +
            aws_docs_client.list_tools_sync()
        )
        print(f"Found {len(all_tools)} tools")
        
        agent = Agent(tools=all_tools, model=bedrock_model, system_prompt=SYSTEM_PROMPT)

        print("Creating migration guide...")
        response = agent(prompt)
        
        return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## Step 5: Amazon Bedrock AgentCore Configuration using Okta as Authorizer (External IDP)

### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import os

boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")

# Use environment variables or defaults (don't use input() in notebooks)
discovery_url = os.getenv('OKTA_DISCOVERY_URL')
client_id = os.getenv('OKTA_CLIENT_ID')
audience = os.getenv('OKTA_AUDIENCE')

print(f"Discovery URL: {discovery_url}")
print(f"Client ID: {client_id[:4]}****{client_id[-4:]}")  # Masked
print(f"Audience: {audience}")

agentcore_runtime = Runtime()

# Try with OAuth configuration
try:
    response = agentcore_runtime.configure(
        entrypoint="my_mcp_agent.py",  # Correct filename
        auto_create_execution_role=True,
        auto_create_ecr=True,
        requirements_file="requirements.txt",
        region=region,
        agent_name="mcp_agent_okta_authfri",  # Consistent naming
        authorizer_configuration={
            "customJWTAuthorizer": {
                "discoveryUrl": discovery_url,
                "allowedClients": [client_id],
                "allowedAudience": [audience]  # Must be a list, not string
            }
        }
    )
    print("✅ OAuth configuration successful")
except Exception as e:
    print(f"❌ OAuth configuration failed: {e}")
    print("Trying without OAuth configuration...")
    
    # Fallback without OAuth
    response = agentcore_runtime.configure(
        entrypoint="my_mcp_agent.py",
        auto_create_execution_role=True,
        auto_create_ecr=True,
        requirements_file="requirements.txt",
        region=region,
        agent_name="mcp_agent_okta_auth"
    )
    print("✅ Basic configuration successful (OAuth can be added later)")

response


### Step 6: Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### Step 7: Checking for the AgentCore Runtime Status

Now that we've deployed the AgentCore Runtime, let's check for its deployment status

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Step 8: Save Agent ARN for Testing

Extract and save the Agent ARN in environment variable

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import os
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using region: {region}")

In [ ]:
import os

# Extract ARN from launch_result (available immediately after launch)
if hasattr(launch_result, 'agent_arn') and launch_result.agent_arn:
    agent_arn = launch_result.agent_arn
    os.environ['AGENT_ARN'] = agent_arn
    print(f"📝 Agent ARN: {agent_arn}")
    print(f"📝 Agent ID: {launch_result.agent_id}")
    print(f"📝 ECR URI: {launch_result.ecr_uri}")
else:
    print("⚠️  Could not extract Agent ARN from launch result")
    print("Launch result:", launch_result)

# Also check final deployment status
if status == 'READY':
    print("✅ Agent deployed successfully and ready for testing!")
elif status in ['CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']:
    print(f"❌ Agent deployment failed with status: {status}")
    print("Check the status response for error details")
else:
    print(f"⚠️  Unexpected status: {status}")

## Step 9: Create Test Client

Now lets create a test client to validate the Okta OAuth flow and agent invocation.

In [ ]:
import requests
import json
import time
import urllib.parse
import logging
import os

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuration
client_id = os.getenv('OKTA_CLIENT_ID')
client_secret = os.getenv('OKTA_CLIENT_SECRET')
audience = os.getenv('OKTA_AUDIENCE')
token_url = os.getenv('OKTA_TOKEN_URL')
AGENT_ARN = os.getenv('AGENT_ARN')

print(f"Client ID: {client_id}")
print(f"Token URL: {token_url}")
print(f"Agent ARN: {AGENT_ARN}")
print(f"token URL: {token_url}")

Function to generate oauth request from Okta 

In [ ]:
def get_oauth_token():
    """Get OAuth token from Okta"""
    data = {
        'grant_type': 'client_credentials',
        'scope': 'agentcore'
    }
    
    logger.info("🔐 Getting OAuth token...")
    
    response = requests.post(
        token_url,
        data=data,
        auth=(client_id, client_secret)
    )
    
    response.raise_for_status()
    token_data = response.json()
    logger.info("✅ OAuth token obtained")
    return token_data['access_token']

Function to invoke Bedrockagentcore Runtime

In [ ]:
def invoke_agent(access_token, query):
    """Invoke MCP agent"""
    escaped_agent_arn = urllib.parse.quote(AGENT_ARN, safe='')
    url = f"https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
    
    session_id = f'test-session-{int(time.time())}-xxxxxxxxxx'
    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id
    }
    
    payload = {'prompt': query}
    
    logger.info(f"🚀 Invoking agent with query: {query}")
    
    response = requests.post(url, headers=headers, json=payload, timeout=300)
    response.raise_for_status()
    
    result = response.json()
    logger.info("✅ Agent response received")
    return result

In [ ]:
# Get OAuth token
access_token = get_oauth_token()
print(f"Token obtained from OKTA: {access_token[:20]}...")

In [ ]:
# Test 1: Simple query to test the agent
print("=" * 50)
print("TEST 1: Simple Query")
print("=" * 50)

result1 = invoke_agent(access_token, "Hello, can you help me?")
print(json.dumps(result1, indent=2))

In [ ]:
# Test 2: Query which will use AWS MCP doc server to answer
print("=" * 50)
print("TEST 2: MCP Query")
print("=" * 50)

result2 = invoke_agent(access_token, "What is the best practices for RDS Aurora MySQL DR?")
print(json.dumps(result2, indent=2))

## Troubleshooting

### Common Issues and Solutions

#### 1. Authentication Errors
```
Error: Failed to get token: 401
```
**Solution**: Verify Okta credentials and configuration
- Check `OKTA_CLIENT_ID` and `OKTA_CLIENT_SECRET`
- Ensure the Okta application has correct grant types
- Verify the token URL format

#### 2. Agent ARN Not Found
```
Error: AGENT_ARN environment variable is required
```
**Solution**: Set the correct Agent ARN from deployment output

#### 3. MCP Server Connection Issues
```
Error: Failed to connect to MCP server
```
**Solution**: 
- Ensure `uvx` is installed in the runtime environment
- Check network connectivity for MCP server downloads
- Verify MCP server package names

#### 4. Permission Errors
```
Error: 403 Forbidden
```
**Solution**: 
- Check Bedrock Agent Core IAM permissions
- Verify OAuth audience and client configuration
- Ensure the JWT token has correct scopes

### Debug Mode

Enable verbose logging for detailed troubleshooting:

In [ ]:
import logging

# Enable debug logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger("urllib3.connectionpool").setLevel(logging.DEBUG)

print("🔍 Debug mode enabled - run your test again for detailed logs")